In [5]:
import sys

import pandas as pd
import numpy as np
import numdifftools as ndt
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from scipy.stats import gamma
from scipy.stats import norm
from scipy.stats import vonmises
from scipy.stats import multivariate_normal
from scipy.stats import gaussian_kde
from scipy.stats import circstd
from scipy.special import iv
from scipy.special import expit
from scipy.special import logit
from scipy.special import logsumexp
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
from scipy.optimize import LinearConstraint
from scipy.signal import convolve
from scipy.interpolate import interp1d
from scipy.linalg import block_diag

from itertools import product

from math import isclose

from copy import deepcopy

from datetime import datetime

import importlib

import time
import pickle
import random

import stoch_optimizor
import optimizor
import HHMM

from helper_funcs import log_delta_2_eta0
from helper_funcs import log_Gamma_2_eta
from helper_funcs import eta0_2_log_delta
from helper_funcs import eta_2_log_Gamma
from helper_funcs import logdotexp

In [30]:
# parse command-line args
max_time = 600*1
id = 1

method_partialEs = [("control",0.0),
                    ("BFGS",0.0),
                    ("CG",0.0),
                    ("GD",0.0),
                    ("SAGA",0.0),
                    ("SAGA",0.5),
                    ("SAGA",1.0),
                    ("SVRG",0.0),
                    ("SVRG",0.5),
                    ("SVRG",1.0)]

rand_seed = [0,1,2,3,4,5,6,7,8,9]

# set methods
for i,settings0 in enumerate(product(rand_seed,method_partialEs)):
    if i == id:
        settings = settings0
        break

rand_seed = settings[0]
method = settings[1][0]
partial_E = settings[1][1]

random.seed(rand_seed)
np.random.seed(rand_seed)

print("method: %s" % method)
print("partial E_step: %.1f" % partial_E)
print("random seed: %d" % rand_seed)
print("max time : %.3f hours" % (max_time/3600))

# select parameters for optimization
num_epochs = 1000
tol = 1e-16
grad_tol = 1e-16

step_sizes = {"EM"  : [None,None],
              "CG"  : [None,None],
              "BFGS": [None,None],
              "GD"  : [0.005,0.005],
              "SGD" : [0.005,0.005],
              "SAG" : [0.005,0.005],
              "SVRG": [0.005,0.005],
              "SAGA": [0.005,0.005]}

jump_every = 1

### checks on optimization parameters ###
if partial_E > 0 and method in ["EM","BFGS","Nelder-Mead","CG"]:
    raise("partial_E not consistent with method")

### features of data ###
features = {'diveDuration'     : {'f'           : 'normal',
                                   'lower_bound' : None,
                                   'upper_bound' : None,
                                   'share_coarse': True,
                                   'share_fine'  : False},
             'maxDepth'         : {'f'           : 'normal',
                                   'lower_bound' : np.array([-np.infty,np.log(5),np.log(5)]),
                                   'upper_bound' : np.array([np.log(20),np.infty,np.infty]),
                                   'share_coarse': True,
                                   'share_fine'  : False},
             'avg_bot_htv'      : {'f'           : 'normal',
                                   'lower_bound' : None,
                                   'upper_bound' : None,
                                   'share_coarse': True,
                                   'share_fine'  : False}}



### load in data ###

# load in data
df = pd.read_csv("../../dat/Final_Data.csv")

# only take whale I107
#whales = ["I145","I107"]
#df = df[df["ID"].isin(whales)]

# convert times
df["stime"] = pd.to_datetime(df["stime"])
df["etime"] = pd.to_datetime(df["etime"])

# force dives to be at least 2 seconds long
df = df[df["diveDuration"] > np.log(2.0)]

# replace -inf
df["max_bot_jp"][df["max_bot_jp"] == -np.infty] = np.NAN

df["broadDiveType"] = np.NAN #3  # unknown
df.loc[df["maxDepth"] > np.log(20),"broadDiveType"] = 1  # deep
df.loc[df["maxDepth"] < np.log(5),"broadDiveType"] = 0  # shallow

# populate a data object

data = []

initial_ts = [0]
final_ts = []

for t,row in enumerate(df.iterrows()):
    
    if t != 0 and df.iloc[t]["ID"] != df.iloc[t-1]["ID"]:
        final_ts.append(t-1)
        initial_ts.append(t)
     
    data.append({"diveDuration"     : row[1]["diveDuration"],
                 "maxDepth"         : row[1]["maxDepth"],
                 "avg_bot_htv"      : row[1]["avg_bot_htv"]})#,
                 #"avg_bot_abs_roll" : row[1]["avg_bot_abs_roll"]})
                 #"postDiveInt"      : np.log(row[1]["postDiveInt"])})
                 #"max_bot_jp"       : row[1]["max_bot_jp"]})#,
                 #"foraging"         : row[1]["foraging"],
                 #"scales"           : row[1]["scales"],
                 #"click.train"      : row[1]["click.train"],
                 #"whales"           : row[1]["whales"]})
                
final_ts.append(t)

initial_ts = np.array(initial_ts)
final_ts = np.array(final_ts)

method: BFGS
partial E_step: 0.0
random seed: 0
max time : 0.167 hours


In [31]:
Ts = [1e3,1e4,1e5]
Ks = [3,5,10]
ds = [1,4,10]
rand_seed = [0,1,2,3,4,5,6,7,8,9]

id = 100

method_partialEs = [("control",0.0),
                    ("BFGS",0.0),
                    ("CG",0.0),
                    ("GD",0.0),
                    ("SAGA",0.0),
                    ("SAGA",0.5),
                    ("SAGA",1.0),
                    ("SVRG",0.0),
                    ("SVRG",0.5),
                    ("SVRG",1.0)]

# set methods
settings_star = (1000.0, 3, 1, 0, ('BFGS', 0.0))
i_star = 10
for i,settings in enumerate(product(Ts,Ks,ds,rand_seed,method_partialEs)):
    if settings == settings_star:#i == i_star:
        print(settings)
        print(i)
        break

(1000.0, 3, 1, 0, ('BFGS', 0.0))
1


In [38]:
from numpy import array
importlib.reload(HHMM)
importlib.reload(optimizor)
importlib.reload(stoch_optimizor)
T = len(data)
K = [2,3]

# pick intial parameters
optim = stoch_optimizor.StochOptimizor(data,features,K)

optim.initial_ts = initial_ts
optim.final_ts = final_ts

for feature in features:
    optim.param_bounds[feature] = {}
    optim.param_bounds[feature]["mu"] = [-100,100]
    optim.param_bounds[feature]["log_sig"] = [-5,5]

if method == "control":
    optim.step_size = step_sizes["SAGA"]
    if not (step_sizes["SAGA"][0] is None):
        optim.L_theta = 1.0 / step_sizes["SAGA"][0]
        optim.L_eta = 1.0 / step_sizes["SAGA"][1]
else:
    optim.step_size = step_sizes[method]
    if not (step_sizes[method][0] is None):
        optim.L_theta = 1.0 / step_sizes[method][0]
        optim.L_eta = 1.0 / step_sizes[method][1]

# print initial parameters
print("initial theta:")
print(optim.theta)
print("")
print("initial eta0:")
print(optim.eta0)
print("")
print("initial eta:")
print(optim.eta)
print("")

optim.theta = [{'diveDuration': {'mu': array([2.93665147, 4.71055169, 4.98305885]), 
                                 'log_sig': array([-0.54630145, -0.1220301 , -0.85076233])}, 
                'maxDepth': {'mu': array([0.68915082, 1.12681522, 2.13726734]), 
                             'log_sig': array([-0.61305886,  0.88465578, -0.1159796 ])}, 
                'avg_bot_htv': {'mu': array([-1.94030359, -0.98937075, -2.20728431]), 
                                'log_sig': array([-0.57657836, -0.59263875, -0.94045475])}}, 
               {'diveDuration': {'mu': array([2.93665147, 4.71055169, 4.98305885]), 
                                 'log_sig': array([-0.54630145, -0.1220301 , -0.85076233])}, 
                'maxDepth': {'mu': array([0.68915082, 1.12681522, 2.13726734]), 
                             'log_sig': array([-0.61305886,  0.88465578, -0.1159796 ])}, 
                'avg_bot_htv': {'mu': array([-1.94030359, -0.98937075, -2.20728431]), 
                                'log_sig': array([-0.57657836, -0.59263875, -0.94045475])}}]
optim.eta = [array([[ 0.        , -5.56375836],
                    [-5.69695586,  0.        ]]),
             [array([[  0.        ,  -1.72271983,  -4.60588735],
                     [  5.65621117,   0.        , -14.09841172],
                     [132.32424276, -33.68535577,   0.        ]]),
              array([[  0.        , -76.62103969,  -3.01498243],
                     [ 58.97936668,   0.        ,  -6.40284949],
                     [  4.85275022, -28.07081821,   0.        ]])]]
optim.eta0 = [array([ 0.       , 57.1931815]),
             [array([ 0.        , 14.34455776, 22.88665213]),
              array([  0.        , -10.02607645,   1.54587799])]]

initial theta:
[{'diveDuration': {'mu': array([2.08176123, 3.50505682, 2.82435693]), 'log_sig': array([-0.26511881,  0.05016391, -0.15086752])}, 'maxDepth': {'mu': array([0.3250374 , 0.82833501, 1.28573821]), 'log_sig': array([-0.28754988,  0.24958784, -0.33550861])}, 'avg_bot_htv': {'mu': array([-2.41852628, -2.5179444 , -1.18210512]), 'log_sig': array([-0.81706676, -1.22717053,  0.29139662])}}, {'diveDuration': {'mu': array([2.08176123, 3.50505682, 2.82435693]), 'log_sig': array([-0.26511881,  0.05016391, -0.15086752])}, 'maxDepth': {'mu': array([0.3250374 , 0.82833501, 1.28573821]), 'log_sig': array([-0.28754988,  0.24958784, -0.33550861])}, 'avg_bot_htv': {'mu': array([-2.41852628, -2.5179444 , -1.18210512]), 'log_sig': array([-0.81706676, -1.22717053,  0.29139662])}}]

initial eta0:
[array([ 0.        , -0.18805628]), [array([ 0.        ,  1.24678097, -0.25388358]), array([ 0.        , -0.87017597,  0.01965374])]]

initial eta:
[array([[ 0.        , -1.35743819],
       [-2.247860

In [42]:
optim.grad_theta

[{'diveDuration': {'mu': array([-6182.34436725,   -41.94597678,  -172.4501021 ]),
   'log_sig': array([4150.74413637,   44.8797693 ,  170.65103779])},
  'maxDepth': {'mu': array([3034.87858668,  -20.13948134,   64.6164737 ]),
   'log_sig': array([-2943.76620533,  -115.24386035,    -5.3481626 ])},
  'avg_bot_htv': {'mu': array([431.32085316, -19.89844301, 173.25506811]),
   'log_sig': array([1778.79684187,    4.68956034,   71.8205245 ])}},
 {'diveDuration': {'mu': array([-6182.34436725,   -41.94597678,  -172.4501021 ]),
   'log_sig': array([4150.74413637,   44.8797693 ,  170.65103779])},
  'maxDepth': {'mu': array([3034.87858668,  -20.13948134,   64.6164737 ]),
   'log_sig': array([-2943.76620533,  -115.24386035,    -5.3481626 ])},
  'avg_bot_htv': {'mu': array([431.32085316, -19.89844301, 173.25506811]),
   'log_sig': array([1778.79684187,    4.68956034,   71.8205245 ])}}]

In [40]:
optim.get_log_like()

-24978.11973202034

# check gradient accuracy

In [37]:
optim.initial_ts

array([   0,  355, 1204, 2733, 3296, 4557, 5040, 5473, 6364, 6837, 7128])

In [35]:
optim.E_step()

In [36]:
display(optim.grad_theta)
#display(optim.grad_theta_t[0])
#display(optim.grad_theta_t[10])

[{'diveDuration': {'mu': array([-6175.62001422,   -34.8783799 ,   -33.51125579]),
   'log_sig': array([4168.22462426,  -19.87932722,   24.36162576])},
  'maxDepth': {'mu': array([3028.72439757,  -36.73807917,   39.22501822]),
   'log_sig': array([-2944.19442698,  -211.62930716,   -41.30977281])},
  'avg_bot_htv': {'mu': array([ 403.56381263, -377.37381318, -158.86016066]),
   'log_sig': array([1771.47502576,  208.24356039,  -33.07950297])}},
 {'diveDuration': {'mu': array([-6175.62001422,   -34.8783799 ,   -33.51125579]),
   'log_sig': array([4168.22462426,  -19.87932722,   24.36162576])},
  'maxDepth': {'mu': array([3028.72439757,  -36.73807917,   39.22501822]),
   'log_sig': array([-2944.19442698,  -211.62930716,   -41.30977281])},
  'avg_bot_htv': {'mu': array([ 403.56381263, -377.37381318, -158.86016066]),
   'log_sig': array([1771.47502576,  208.24356039,  -33.07950297])}}]

In [28]:
x0 = optim.params_2_x()

# get log likelihood to find hessian indices
def get_ll(x):

    print(x-x0)

    optim.x_2_params(x)

    for t in range(optim.T):
        optim.update_alpha(t)

    optim.x_2_params(x0)

    return logsumexp(optim.log_alphas[-1])

# get indices where hessian nonnegative
jacobian = ndt.Jacobian(get_ll,step=0.00001)(x0)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[-1.e-05  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00]
[0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00

[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00 -1.e-05  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00]
[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+

[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00 -1.e-05  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00]
[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -1.e-05  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00

[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -1.e-05  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00]
[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e-05 0.e+00 0.e+

[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
 -1.e-05  0.e+00  0.e+00]
[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e-05 0.e+00]
[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00

In [29]:
display(jacobian)
display(optim.grad_params_2_xprime())

array([[ 1.67964959e+02,  1.00341308e+02,  1.72556460e+02,
        -1.32370367e+03,  2.29618929e+02,  5.48164313e+02,
         6.37884166e+02,  1.66721117e+02,  3.92395596e+01,
        -9.71123056e+02,  8.76963784e+02,  2.35176063e+02,
        -1.75673702e+03,  1.01215520e+02,  1.51410504e+01,
         8.96608024e+02,  1.73285362e+02, -1.44199173e+01,
         0.00000000e+00, -1.86291855e+01,  4.56516318e-01,
         0.00000000e+00,  0.00000000e+00, -2.25700309e+02,
        -2.56362188e+02,  7.22227273e+01,  0.00000000e+00,
        -8.86200660e+00,  1.96467499e+01, -1.31245106e+00,
         0.00000000e+00,  0.00000000e+00, -5.34571045e+00,
        -2.32205884e+00,  9.56263921e-01,  0.00000000e+00,
        -5.27898646e-01,  2.83715872e-01, -9.42323823e-02,
         0.00000000e+00,  0.00000000e+00, -3.29284194e-01,
         0.00000000e+00, -9.12118003e+00, -4.81187435e+00,
         0.00000000e+00, -3.19809587e+00, -7.61704150e+00]])

array([ 1.67964959e+02,  1.00341308e+02,  1.72556461e+02, -1.32370367e+03,
        2.29618929e+02,  5.48164313e+02,  6.37884167e+02,  1.66721118e+02,
        3.92395595e+01, -9.71123056e+02,  8.76963784e+02,  2.35176062e+02,
       -1.75673702e+03,  1.01215521e+02,  1.51410498e+01,  8.96608024e+02,
        1.73285362e+02, -1.44199177e+01,  0.00000000e+00, -1.86292046e+01,
        4.56517906e-01,  0.00000000e+00,  0.00000000e+00, -2.25700308e+02,
       -2.56362186e+02,  7.22227255e+01,  0.00000000e+00, -8.86200646e+00,
        1.96467437e+01, -1.31247490e+00,  0.00000000e+00,  0.00000000e+00,
       -5.34570886e+00, -2.32205930e+00,  9.56263585e-01,  0.00000000e+00,
       -5.27897075e-01,  2.83716247e-01, -9.42315653e-02,  0.00000000e+00,
        0.00000000e+00, -3.29284189e-01,  0.00000000e+00, -9.12118229e+00,
       -4.81187651e+00,  0.00000000e+00, -3.19809582e+00, -7.61704002e+00])

# make sure getting likelihood doesn't change alpha, beta, or grads

In [22]:
t = 10

print(optim.log_alphas[t])
print(optim.log_betas[t])
print(optim.p_Xt[t])
print(optim.p_Xtm1_Xt[t])
print("")
print(optim.get_ll_keep_params())
print("")
print(optim.log_alphas[t])
print(optim.log_betas[t])
print(optim.p_Xt[t])
print(optim.p_Xtm1_Xt[t])

[-70.24670553         -inf         -inf -73.3144889          -inf
         -inf]
[-1833.17238644 -1834.18703886 -1834.16840313 -1833.76284113
 -1834.39243788 -1834.36258983]
[0.97486998 0.         0.         0.02513002 0.         0.        ]
[[0.96365819 0.         0.         0.01162068 0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.01121179 0.         0.         0.01350933 0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]]

(-1903.393635120825, 493.38175775668003)

[-70.24670553         -inf         -inf -73.3144889          -inf
         -inf]
[-1833.17238644 -1834.18703886 -1834.16840313 -1833.76284113
 -1834.39243788 -1834.36258983]
[0.97486998 0.         0.         0.02513002 0.         0.        ]
[[0.96365819 0.         0.         0.01162068 0.         0

In [15]:
print(optim.get_ll_keep_params())

(-1903.393635120825, 493.38175775668003)


In [16]:
print(optim.log_alphas)
print(optim.log_betas)

[[          -inf   -22.25104669    -9.1688989            -inf
    -21.61126398    -7.19286225]
 [  -12.28295854           -inf           -inf   -13.19964103
            -inf           -inf]
 [  -22.55078179           -inf           -inf   -24.11616785
            -inf           -inf]
 ...
 [-1894.78128944           -inf           -inf -1897.67465727
            -inf           -inf]
 [-1898.61087319           -inf           -inf -1901.61054691
            -inf           -inf]
 [-1906.75339424 -1904.9015275  -1903.76890359 -1909.81081662
  -1907.66162282 -1906.58454738]]
[[-1895.15999008 -1896.17541934 -1896.15676111 -1895.76914992
  -1896.3863357  -1896.35726432]
 [-1891.32687667 -1892.33779096 -1892.31926331 -1891.83172819
  -1892.51711629 -1892.4836632 ]
 [-1880.9836884  -1881.64545591 -1879.66742142 -1881.30726834
  -1880.54582939 -1880.42765649]
 ...
 [   -8.6531283     -9.65295641    -9.63474697    -8.93874574
     -9.75925037    -9.71624907]
 [   -4.83245729    -4.72159159    -3.7